## Install dependencies


gymnasium: Contains the LunarLander-v2 environment 🌛
stable-baselines3: The deep reinforcement learning library.
huggingface_sb3: Additional code for Stable-baselines3 to load and upload models from the Hugging Face 🤗 Hub.
All included in the

In [ ]:
!apt install swig cmake

In [ ]:
!pip install -r https://raw.githubusercontent.com/huggingface/deep-rl-class/main/notebooks/unit1/requirements-unit1.txt

In [ ]:
!sudo apt-get update
!apt install python3-opengl
!apt install ffmpeg
!apt install xvfb
!pip3 install pyvirtualdisplay

In [4]:
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

In [5]:
import gymnasium

from huggingface_sb3 import load_from_hub, package_to_hub
from huggingface_hub import (
    notebook_login,
)  # To log to our Hugging Face account to be able to upload models to the Hub.

from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor

In [7]:
import gymnasium as gym

# First, we create our environment called LunarLander-v2
env = gym.make("LunarLander-v2")

# Then we reset this environment
observation, info = env.reset()

for _ in range(20):
    # Take a random action
    action = env.action_space.sample()
    print("Action taken:", action)

    # Do this action in the environment and get
    # next_state, reward, terminated, truncated and info
    observation, reward, terminated, truncated, info = env.step(action)

    # If the game is terminated (in our case we land, crashed) or truncated (timeout)
    if terminated or truncated:
        # Reset the environment
        print("Environment is reset")
        observation, info = env.reset()

env.close()

Action taken: 2
Action taken: 2
Action taken: 3
Action taken: 1
Action taken: 1
Action taken: 2
Action taken: 2
Action taken: 2
Action taken: 1
Action taken: 2
Action taken: 3
Action taken: 0
Action taken: 1
Action taken: 2
Action taken: 1
Action taken: 1
Action taken: 2
Action taken: 3
Action taken: 1
Action taken: 0


In [8]:
env = gym.make("LunarLander-v2")
env.reset()
print("_____OBSERVATION SPACE_____ \n")
print("Observation Space Shape", env.observation_space.shape)
print("Sample observation", env.observation_space.sample())  # Get a random observation

_____OBSERVATION SPACE_____ 

Observation Space Shape (8,)
Sample observation [-54.371525    37.49116     -2.1827085   -0.63440627  -2.025829
   3.5611644    0.3409257    0.4477141 ]


In [9]:
print("\n _____ACTION SPACE_____ \n")
print("Action Space Shape", env.action_space.n)
print("Action Space Sample", env.action_space.sample())  # Take a random action


 _____ACTION SPACE_____ 

Action Space Shape 4
Action Space Sample 2


In [10]:
# Create the Vectorized environment
env = make_vec_env("LunarLander-v2", n_envs=16)

In [12]:
model = PPO(
    policy="MlpPolicy",
    env=env,
    n_steps=1024,
    batch_size=64,
    n_epochs=4,
    gamma=0.999,
    gae_lambda=0.98,
    ent_coef=0.01,
    verbose=1,
)

Using cuda device


In [13]:
# Train it for 1,000,000 timesteps
model.learn(total_timesteps=1000000)
# Save the model
model_name = "ppo-LunarLander-v2"
model.save(model_name)

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 90.6     |
|    ep_rew_mean     | -193     |
| time/              |          |
|    fps             | 4211     |
|    iterations      | 1        |
|    time_elapsed    | 3        |
|    total_timesteps | 16384    |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 97.2         |
|    ep_rew_mean          | -158         |
| time/                   |              |
|    fps                  | 2574         |
|    iterations           | 2            |
|    time_elapsed         | 12           |
|    total_timesteps      | 32768        |
| train/                  |              |
|    approx_kl            | 0.0056646196 |
|    clip_fraction        | 0.0339       |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.38        |
|    explained_variance   | 0.000492     |
|    learning_r

In [14]:
# Create a new environment for evaluation
eval_env = Monitor(gym.make("LunarLander-v2"))

# Evaluate the model with 10 evaluation episodes and deterministic=True
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=10, deterministic=True)

# Print the results
print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")

mean_reward=260.12 +/- 17.767067599411398


In [15]:
notebook_login()

In [20]:
# method save, evaluate, generate a model card and record a replay video of your agent before pushing the repo to the hub
package_to_hub(model=model, # Our trained model
               model_name=model_name,
               model_architecture="PPO", # The model architecture we used: in our case PPO
               env_id="LunarLander-v2",
               eval_env=eval_env, # Evaluation Environment
               repo_id="mohamedsaeed823/ppo-LunarLander-v2", # id of the model repository from the Hugging Face Hub (repo_id = {organization}/{repo_name} for instance ThomasSimonini/ppo-LunarLander-v2
               commit_message="package-to-hub the agent",
               )

ℹ This function will save, evaluate, generate a video of your agent,
create a model card and push everything to the hub. It might take up to 1min.
This is a work in progress: if you encounter a bug, please open an issue.
Saving video to /tmp/tmpv2rmwdg_/-step-0-to-step-1000.mp4
Moviepy - Building video /tmp/tmpv2rmwdg_/-step-0-to-step-1000.mp4.
Moviepy - Writing video /tmp/tmpv2rmwdg_/-step-0-to-step-1000.mp4



Moviepy - Done !
Moviepy - video ready /tmp/tmpv2rmwdg_/-step-0-to-step-1000.mp4
ℹ Pushing repo mohamedsaeed823/ppo-LunarLander-v2 to the Hugging Face
Hub


policy.pth:   0%|          | 0.00/43.8k [00:00<?, ?B/s]

policy.optimizer.pth:   0%|          | 0.00/88.4k [00:00<?, ?B/s]

pytorch_variables.pth:   0%|          | 0.00/864 [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

ppo-LunarLander-v2.zip:   0%|          | 0.00/148k [00:00<?, ?B/s]

ℹ Your model is pushed to the Hub. You can view your model here:
https://huggingface.co/mohamedsaeed823/ppo-LunarLander-v2/tree/main/


CommitInfo(commit_url='https://huggingface.co/mohamedsaeed823/ppo-LunarLander-v2/commit/49cce548c0f41a70e7aa6c1e676c1a3d6bd2a472', commit_message='package-to-hub the agent', commit_description='', oid='49cce548c0f41a70e7aa6c1e676c1a3d6bd2a472', pr_url=None, pr_revision=None, pr_num=None)